# Budowanie pytań do Youtube
Wszystkie zapytania do Youtube będziemy poświadczać przez nasz klucz API uzyskany w konsoli Google Cloud. Zapiszemy go do zmiennej w pliku.

In [ ]:
DEVELOPER_KEY = ""

 ## Poszukujemy ID kanału.

Przechodzimy do [dokumentacji Youtube](https://developers.google.com/youtube/v3/docs/search/list) - na ostatnich zajęciach korzystaliśmy z opcji automatycznego generowania adresu HTTP, który przekazywaliśmy do Excela albo Postman. Było to jednak trochę uciążliwe. 

Ostatnia karta zawiera pełen kod w języku Python - w praktyce jest on dość długi. Interesują nas 4 ostatnie linie kodu i zmiana sposobu uwierzytelniania. 

Przedstawie je poniżej: 

In [ ]:
import googleapiclient.discovery
youtube = googleapiclient.discovery.build("youtube", "v3", 
                                          developerKey= DEVELOPER_KEY)
request = youtube.search().list(
    part="snippet",
    q="Radio Zet", # ZMIENIAJ TĄ LINIJKĘ KODU JEŻELI CHCESZ ZAPYTAĆ O INNY KANAŁ
    type="channel")

response = request.execute()
response

Możemy też zmienić formę na nieco bardziej czytelną:

In [ ]:
pd.json_normalize(response)

Wynik ma bardzo nieprzystępną formę. Ale możemy to szybko zmienić wykorzystujac bibliotekę *pandas*. W zasadzie potrzebujemy znać dwie komendy: 

1. Najppierw przechodzimy do części *items* z naszej ramki zawierającej wszelkie dane:

In [ ]:
import pandas as pd
dane = response["items"] # WYBIERZ DANE Z POZYCJI ITEMS 
response_data_df = pd.DataFrame(dane) # PRZEKSZTAŁC TO W FORMAT ARKUSZA I ZAPISZ DO ZMIENNEJ response_data_df 
response_data_df # WYDRUKUJ RAMKE

2. Następnie rozpakowujemy wynik z id, albo pobrany snippet. Albo jedno i drugie:

In [ ]:
df2 = pd.json_normalize(response_data_df['id']) # PRZEKSZTAŁC CZĘŚĆ ID DO CZYTELNYCH DANYCH
df3 = pd.json_normalize(response_data_df['snippet']) # PRZEKSZTAŁC CZĘŚĆ SNIPPET DO CZYTELNYCH DANYCH
wynik =  pd.concat([df2, df3], axis=1) # ZLACZ CZESCI ID i SNIPPET W JEDNA RAMKE

Treść mozemy odczytać wpisując nazwę zmiennej:

In [ ]:
df3

Możemy wybrać nazwy kolumn, aby wszystko stało się prostsze:

In [ ]:
wynik[["channelId", "title", "description"]] # WYBIERZ KOLUMNY ChannelID, title I description

## Pobieramy statystyki Video - Szukamy filmów
Na poprzednich zajęciach szukaliśmy playlist i na ich podstawie określaliśmy video będące na kanale. 

Komendę *search*, którą wykrozystaliśmy do znalezienia ID kanału można wykorzystać też do przeszukania video spełniających określone kryteria.

In [ ]:
request = youtube.search().list(
    part="snippet",
    channelId = "UCvHFbkohgX29NhaUtmkzLmg", # TEN ARGUMENT OGRANICZA PRZESZUKIWANIA DO JEDNEGO KANALU
    q="Podejrzani Politycy", # ZMIENIAJ TĄ LINIJKĘ KODU JEŻELI CHCESZ ZAPYTAĆ O INNE VIDEO
    type="video") # TYPE ZMIENIL SIE Z CHANNEL NA VIDEO - TERAZ SZUKAMY FILMOW

response = request.execute()
pd.json_normalize(response)

Ten krótki kod przekształca dane do czytelnego obrazku:

In [ ]:
dane = response["items"] # WYBIERZ DANE Z POZYCJI ITEMS 
response_data_df = pd.DataFrame(dane) # PRZEKSZTAŁC TO W FORMAT ARKUSZA I ZAPISZ DO ZMIENNEJ response_data_df 
df2 = pd.json_normalize(response_data_df['id']) # PRZEKSZTAŁC CZĘŚĆ ID DO CZYTELNYCH DANYCH
df3 = pd.json_normalize(response_data_df['snippet']) # PRZEKSZTAŁC CZĘŚĆ SNIPPET DO CZYTELNYCH DANYCH
wynik =  pd.concat([df2, df3], axis=1) # ZLACZ CZESCI ID i SNIPPET W JEDNA RAMKE
wynik[["videoId", "title", "description", "publishTime"]] # WYBIERZ KOLUMNY

Mamy tylko 5 rekordów - to mało. Możemy zwiększyć te liczbę do 50. Wymaga to dodatkowej komendy w poleceniu z pobieraniem danych. My wykorzystamy ją, aby pobrać 10 rekordów.  

In [ ]:
request = youtube.search().list(
    part="snippet",
    channelId = "UCvHFbkohgX29NhaUtmkzLmg", 
    q="Podejrzani Politycy", 
    maxResults = 10, # DODAJE TA LINIJKE ABY WYBRAC LICZBE WYNIKOW - MOZEMY MIEC DO 50 WYNIKÓW 
    type="video") 

response = request.execute()
pd.json_normalize(response)

In [ ]:
dane = response["items"]
response_data_df = pd.DataFrame(dane)
df2 = pd.json_normalize(response_data_df['id'])
df3 = pd.json_normalize(response_data_df['snippet'])
wynik =  pd.concat([df2, df3], axis=1) 
lista_filmow = wynik[["videoId", "title", "description", "publishTime"]] # ZAPISUJE WYNIKI DO ZMINNEJ LISTA FILMOW
lista_filmow # POSLUZY NAM ONA DO KOLEJNYCH POSZUKIWAN

## Pobieramy statystyki Video - Pobieramy informacje
Umiemy wybrać dużo rekordów i zebrać ich id filmów. Teraz potrzebne nam statystyki. Pojedynczy film opisujemy następującym blokiem kodu:

In [ ]:
request = youtube.videos().list(
    part ="statistics",
    id ="gD41UFKvGAE"
    )

response = request.execute()
wynik = pd.json_normalize(response["items"])
wynik

Ską wiem jak go zrobić? W dokumentacji wybieram sekcje [Videos](https://developers.google.com/youtube/v3/docs/videos/list) i kopiuje kod na podstawie sugestii Google, podobnie jak to miało miejsce z wynikami szukania.

Teraz potrzebuje zbudować taki kod, aby przeszedł przez wszystkie elementy z moejej listy filmów i pobrał dane. Posłuzy do tego pętla *for*. W przykładzie poinizej wydrukuje wszystkie ID:

In [ ]:
for film in lista_filmow["videoId"].values:
  print(film)

Realnie chcę jednak pobrać informację - musze połączyć kody z poprzednich dwóch listingów - pętle for i pobieranie statystyk. 

In [ ]:
import time 

for film in lista_filmow["videoId"].values:
  request = youtube.videos().list(
      part ="statistics",
      id = film) ## TUTAJ ZAMIAST OKRESLONEGO FILMU POJAWIA SIE NASZ ELEEMNT PENTLI

  response = request.execute()
  tymczasowy_wynik = pd.json_normalize(response["items"])
  wynik = pd.concat([wynik, tymczasowy_wynik]) # LACZE DOTYCHCZASOWY WYNIK Z NOWYM POBIERANIEM

  time.sleep(2) # 2 SEKUNDY PAUZY MIĘDZY POBRANIAMI, ABUY NIE DOSTAC BANA

Zobaczmy nasz wynik:

In [ ]:
wynik

# Paginacja
Co jeśli nasze poszukiwania mają więcej niż 50 rekordów, a chcemy mieć wszystkie dane. W takim wypadku nasze dane są dzielone na cześci. 

Wróćmy do pytania o podejrzanych polityków:

In [ ]:
request = youtube.search().list(
    part="snippet",
    channelId = "UCvHFbkohgX29NhaUtmkzLmg", # TEN ARGUMENT OGRANICZA PRZESZUKIWANIA DO JEDNEGO KANALU
    q="Podejrzani Politycy", # ZMIENIAJ TĄ LINIJKĘ KODU JEŻELI CHCESZ ZAPYTAĆ O INNE VIDEO
    type="video") # TYPE ZMIENIL SIE Z CHANNEL NA VIDEO - TERAZ SZUKAMY FILMOW

response = request.execute()
pd.json_normalize(response)

Tak wygląda pierwszych 5 rekordów:

In [ ]:
dane = response["items"] # WYBIERZ DANE Z POZYCJI ITEMS 
response_data_df = pd.DataFrame(dane) # PRZEKSZTAŁC TO W FORMAT ARKUSZA I ZAPISZ DO ZMIENNEJ response_data_df 
df2 = pd.json_normalize(response_data_df['id']) # PRZEKSZTAŁC CZĘŚĆ ID DO CZYTELNYCH DANYCH
df3 = pd.json_normalize(response_data_df['snippet']) # PRZEKSZTAŁC CZĘŚĆ SNIPPET DO CZYTELNYCH DANYCH
wynik =  pd.concat([df2, df3], axis=1) # ZLACZ CZESCI ID i SNIPPET W JEDNA RAMKE
wynik[["videoId", "title", "description", "publishTime"]] # WYBIERZ KOLUMNY

Aby wyświetlić kolejne wiersze do pierwszego pytania dodajemy kolejny argument *pageToken*. Przyjmie on wartość z pozycji *nextPageToken* z poprzedniego wyszukiwania: 

In [ ]:
request = youtube.search().list(
    part="snippet",
    channelId = "UCvHFbkohgX29NhaUtmkzLmg", 
    q="Podejrzani Politycy", 
    pageToken = "CAUQAA", # TO DODAJEMY ABY WYBRAĆ KOLEJNE 5 FILMOW
    type="video")

response = request.execute()
pd.json_normalize(response)

In [ ]:
dane = response["items"] # WYBIERZ DANE Z POZYCJI ITEMS 
response_data_df = pd.DataFrame(dane) # PRZEKSZTAŁC TO W FORMAT ARKUSZA I ZAPISZ DO ZMIENNEJ response_data_df 
df2 = pd.json_normalize(response_data_df['id']) # PRZEKSZTAŁC CZĘŚĆ ID DO CZYTELNYCH DANYCH
df3 = pd.json_normalize(response_data_df['snippet']) # PRZEKSZTAŁC CZĘŚĆ SNIPPET DO CZYTELNYCH DANYCH
wynik =  pd.concat([df2, df3], axis=1) # ZLACZ CZESCI ID i SNIPPET W JEDNA RAMKE
wynik[["videoId", "title", "description", "publishTime"]] # WYBIERZ KOLUMNY